In [1]:
import grequests, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime
from time import sleep
import pytz
from pyModbusTCP.client import ModbusClient
from IPython.display import clear_output
import signal
from contextlib import contextmanager

PI_string = ['https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwt9OsDGKz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w2NzUwVlw2NzUwLVBQLTAwMQ/value',
             'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwFxaBGGKz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w2NzUwVlw2NzUwLVBQLTAwMg/value',
             'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwJ300KmKz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w2NzUwVlw2NzUwLVBQLTAwMw/value',
             'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwRyG1NWKz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w2NzUwVlw2NzUwLVBQLTAwNA/value']

now = datetime.now()
new_timezone = pytz.timezone("Etc/GMT+4")
client_tambo1 = ModbusClient(host="192.168.168.105", port=1234)
client_tambo1.open()

True

In [2]:
nombre_archivo = 'Tambo1_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)
tambo1_pp1, tambo1_pp2, tambo1_pp3, tambo1_pp4 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])
with pd.ExcelWriter('~/Escritorio/Caserones/datos/Tambo1/'+nombre_archivo) as writer:  
    tambo1_pp1.to_excel(writer,sheet_name='PP-001')
    tambo1_pp2.to_excel(writer,sheet_name='PP-002')
    tambo1_pp3.to_excel(writer,sheet_name='PP-003')
    tambo1_pp4.to_excel(writer,sheet_name='PP-004')

In [3]:
@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        pass
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)

def raise_timeout(signum, frame):
    raise TimeoutError
    
def pedir_datos(pi_string):
    with timeout(30):
        reqs = [grequests.get(pi_string,auth=HTTPBasicAuth('vulco.esed','vpump$mlcc2021'))]
        valores = grequests.map(reqs)
    return valores

In [ ]:
while True:    
    try:
        now = datetime.now()
        if (nombre_archivo != 'Tambo1_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)):
            tambo1_pp1, tambo1_pp2, tambo1_pp3, tambo1_pp4 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])
            
        clear_output(wait=True)
        
        print("El Tambo 1")
        ###
        print('PP-001')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[0])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            flow = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            res_imp = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_dev2C = json.loads(valores[0].text)['Items'][3]['Value']['Value']
            temp_dev1C = json.loads(valores[0].text)['Items'][4]['Value']['Value']
            temp_dev2B = json.loads(valores[0].text)['Items'][5]['Value']['Value']
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value']
            temp_dev2A = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            temp_dev1A = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            temp_descB = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][10]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][11]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][12]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][13]['Value']['Value']
            corr_C = json.loads(valores[0].text)['Items'][14]['Value']['Value']
            corr_B = json.loads(valores[0].text)['Items'][15]['Value']['Value']
            corr_A = json.loads(valores[0].text)['Items'][16]['Value']['Value']
        
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
        
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)
        
            hor_logica, estado, ef_inst, velocidad_motor, corr_total, pres_des = 0, 0, 0, 0, 0, 0 
            velocidad_bomba, temp_bomba, acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z, = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
    
            print("Escribiendo al Modbus")
            client_tambo1.write_multiple_registers(1150,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                               tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100), np.int16(velocidad_motor), 
                                               np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C), np.int16(pot_act),
                                               np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A),
                                               np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB),
                                               np.int16(pres_des), np.int16(flow), np.int16(velocidad_bomba), np.int16(temp_bomba),
                                               np.int16(acel_hor), np.int16(acel_ver), np.int16(acel_ax), np.int16(vel_hor), np.int16(vel_ver), 
                                               np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y), np.int16(p2p_z),
                                               np.int16(id_sensor), np.int16(bateria_sensor), np.int16(rssi_sensor), pi_av])
            
            print("Guardando dataframe")
            pp1 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],
                                 'Horas operando':[hor_op],'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],
                                 'Velocidad motor':[velocidad_motor],'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],
                                 'Corriente C':[corr_C],'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            tambo1_pp1 = tambo1_pp1.append(pp1,ignore_index=True)    

        except:
            print('Conexion no lograda')
            client_tambo1.write_multiple_registers(1190,[0])
            pass
        
        ###
        print('PP-002')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[1])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            flow = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            res_imp = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_dev2C = json.loads(valores[0].text)['Items'][3]['Value']['Value']
            temp_dev1C = json.loads(valores[0].text)['Items'][4]['Value']['Value']
            temp_dev2B = json.loads(valores[0].text)['Items'][5]['Value']['Value']
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value']
            temp_dev2A = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            temp_dev1A = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            temp_descB = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][10]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][11]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][12]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][13]['Value']['Value']
            corr_C = json.loads(valores[0].text)['Items'][14]['Value']['Value']
            corr_B = json.loads(valores[0].text)['Items'][15]['Value']['Value']
            corr_A = json.loads(valores[0].text)['Items'][16]['Value']['Value']
        
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
        
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)
        
            hor_logica, estado, ef_inst, velocidad_motor, corr_total, pres_des = 0, 0, 0, 0, 0, 0 
            velocidad_bomba, temp_bomba, acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z, = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
    
            print("Escribiendo al Modbus")
            client_tambo1.write_multiple_registers(1200,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                               tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100), np.int16(velocidad_motor), 
                                               np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C), np.int16(pot_act),
                                               np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A),
                                               np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB),
                                               np.int16(pres_des), np.int16(flow), np.int16(velocidad_bomba), np.int16(temp_bomba),
                                               np.int16(acel_hor), np.int16(acel_ver), np.int16(acel_ax), np.int16(vel_hor), np.int16(vel_ver), 
                                               np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y), np.int16(p2p_z),
                                               np.int16(id_sensor), np.int16(bateria_sensor), np.int16(rssi_sensor), pi_av])
            
            print("Guardando dataframe")
            pp2 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],
                                 'Horas operando':[hor_op],'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],
                                 'Velocidad motor':[velocidad_motor],'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],
                                 'Corriente C':[corr_C],'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            tambo1_pp2 = tambo1_pp2.append(pp2,ignore_index=True)    

        except:
            print('Conexion no lograda')
            client_tambo1.write_multiple_registers(1240,[0])
            pass
        
        ###
        print('PP-003')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[2])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            flow = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            res_imp = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_dev2C = json.loads(valores[0].text)['Items'][3]['Value']['Value']
            temp_dev1C = json.loads(valores[0].text)['Items'][4]['Value']['Value']
            temp_dev2B = json.loads(valores[0].text)['Items'][5]['Value']['Value']
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value']
            temp_dev2A = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            temp_dev1A = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            temp_descB = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][10]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][11]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][12]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][13]['Value']['Value']
            corr_C = json.loads(valores[0].text)['Items'][14]['Value']['Value']
            corr_B = json.loads(valores[0].text)['Items'][15]['Value']['Value']
            corr_A = json.loads(valores[0].text)['Items'][16]['Value']['Value']
        
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
        
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)
        
            hor_logica, estado, ef_inst, velocidad_motor, corr_total, pres_des = 0, 0, 0, 0, 0, 0 
            velocidad_bomba, temp_bomba, acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z, = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
    
            print("Escribiendo al Modbus")
            client_tambo1.write_multiple_registers(1250,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                               tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100), np.int16(velocidad_motor), 
                                               np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C), np.int16(pot_act),
                                               np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A),
                                               np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB),
                                               np.int16(pres_des), np.int16(flow), np.int16(velocidad_bomba), np.int16(temp_bomba),
                                               np.int16(acel_hor), np.int16(acel_ver), np.int16(acel_ax), np.int16(vel_hor), np.int16(vel_ver), 
                                               np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y), np.int16(p2p_z),
                                               np.int16(id_sensor), np.int16(bateria_sensor), np.int16(rssi_sensor), pi_av])
            
            print("Guardando dataframe")
            pp3 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],
                                 'Horas operando':[hor_op],'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],
                                 'Velocidad motor':[velocidad_motor],'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],
                                 'Corriente C':[corr_C],'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            tambo1_pp3 = tambo1_pp3.append(pp3,ignore_index=True)    

        except:
            print('Conexion no lograda')
            client_tambo1.write_multiple_registers(1290,[0])
            pass

        ###
        print('PP-004')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[3])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            flow = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            res_imp = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_dev2C = json.loads(valores[0].text)['Items'][3]['Value']['Value']
            temp_dev1C = json.loads(valores[0].text)['Items'][4]['Value']['Value']
            temp_dev2B = json.loads(valores[0].text)['Items'][5]['Value']['Value']
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value']
            temp_dev2A = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            temp_dev1A = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            temp_descB = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][10]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][11]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][12]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][13]['Value']['Value']
            corr_C = json.loads(valores[0].text)['Items'][14]['Value']['Value']
            corr_B = json.loads(valores[0].text)['Items'][15]['Value']['Value']
            corr_A = json.loads(valores[0].text)['Items'][16]['Value']['Value']
        
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
        
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)
        
            hor_logica, estado, ef_inst, velocidad_motor, corr_total, pres_des = 0, 0, 0, 0, 0, 0 
            velocidad_bomba, temp_bomba, acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z, = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
    
            print("Escribiendo al Modbus")
            client_tambo1.write_multiple_registers(1300,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                               tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100), np.int16(velocidad_motor), 
                                               np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C), np.int16(pot_act),
                                               np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A),
                                               np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB),
                                               np.int16(pres_des), np.int16(flow), np.int16(velocidad_bomba), np.int16(temp_bomba),
                                               np.int16(acel_hor), np.int16(acel_ver), np.int16(acel_ax), np.int16(vel_hor), np.int16(vel_ver), 
                                               np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y), np.int16(p2p_z),
                                               np.int16(id_sensor), np.int16(bateria_sensor), np.int16(rssi_sensor), pi_av])
            
            print("Guardando dataframe")
            pp4 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],
                                 'Horas operando':[hor_op],'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],
                                 'Velocidad motor':[velocidad_motor],'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],
                                 'Corriente C':[corr_C],'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            tambo1_pp4 = tambo1_pp4.append(pp4,ignore_index=True)    

        except:
            print('Conexion no lograda')
            client_tambo1.write_multiple_registers(1340,[0])
            pass        

        ###
        print("Guardando excel")
        nombre_archivo = 'Tambo1_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)
        with pd.ExcelWriter('~/Escritorio/Caserones/datos/Tambo1/'+nombre_archivo) as writer:  
            tambo1_pp1.to_excel(writer,sheet_name='PP-001')
            tambo1_pp2.to_excel(writer,sheet_name='PP-002')
            tambo1_pp3.to_excel(writer,sheet_name='PP-003')
            tambo1_pp4.to_excel(writer,sheet_name='PP-004')
        sleep(5)

    except:
        now = datetime.now()
        clear_output(wait=True)
        print("Algo fallo, lo intentare de nuevo")
        fecha_error = "Timestamp: {}-{}-{} {}:{}".format(now.year,now.month,now.day,now.hour,now.minute)
        print(fecha_error)
        sleep(5)
        continue

El Tambo 1
PP-001
Solicitando datos


Traceback (most recent call last):
  File "/tmp/ipykernel_41801/911109657.py", line 18, in raise_timeout
    raise TimeoutError
TimeoutError
2021-08-18T14:22:08Z


Conexion no lograda
PP-002
Solicitando datos


Traceback (most recent call last):
  File "/tmp/ipykernel_41801/911109657.py", line 18, in raise_timeout
    raise TimeoutError
TimeoutError
2021-08-18T14:22:38Z


Conexion no lograda
PP-003
Solicitando datos
Escribiendo al Modbus
Guardando dataframe
PP-004
Solicitando datos


Traceback (most recent call last):
  File "/tmp/ipykernel_41801/911109657.py", line 18, in raise_timeout
    raise TimeoutError
TimeoutError
2021-08-18T14:23:29Z


Conexion no lograda
Guardando excel
